In [1]:
import gensim
import numpy as np
import os
import sys

ROOT = os.path.dirname(os.getcwd())
path_data = 'C:\\Users\\Gosia\\Desktop\\'
sys.path.insert(0, ROOT)

# this is the path to the source code of this project
sys.path.append('../src/')

In [2]:
import passatutto_parser as pp
from matchms.importing.load_from_json import as_spectrum


pre_spectrums_query = pp.PassatuttoParser(r'C:\\Users\\Gosia\\Desktop\\MassbankOrbi').parse_folder()
pre_spectrums_lib = pp.PassatuttoParser('C:\\Users\\Gosia\\Desktop\\Gnps').parse_folder()

# Using MatchMS to create spectra for both
spectrums_query = []
for i, s in enumerate( pre_spectrums_query ):
    spectrums_query.append(as_spectrum(s))
    if i and i % 100 == 0:
        print('processed %d' % i)
        
spectrums_lib = []
for i, s in enumerate( pre_spectrums_lib ):
    spectrums_lib.append(as_spectrum(s))
    if i and i % 100 == 0:
        print('processed %d' % i)
        

processed 100 files
processed 200 files
processed 300 files
processed 400 files
Finished parsing of 458 spectra 
processed 100 files
processed 200 files
processed 300 files
processed 400 files
processed 500 files
processed 600 files
processed 700 files
processed 800 files
processed 900 files
processed 1000 files
processed 1100 files
processed 1200 files
processed 1300 files
processed 1400 files
processed 1500 files
processed 1600 files
processed 1700 files
processed 1800 files
processed 1900 files
processed 2000 files
processed 2100 files
processed 2200 files
processed 2300 files
processed 2400 files
processed 2500 files
processed 2600 files
processed 2700 files
processed 2800 files
processed 2900 files
processed 3000 files
processed 3100 files
processed 3200 files
processed 3300 files
processed 3400 files
processed 3500 files
processed 3600 files
processed 3700 files
processed 3800 files
processed 3900 files
processed 4000 files
processed 4100 files
Finished parsing of 4138 spectra 
p

In [ ]:
# Loading the decoy database
pre_spectrums_decoys_cond = pp.DecoyParserPassattuto(r'C:\\Users\\Gosia\\Desktop\\GnpsDecoyConditionalPeaks').parse_folder()
pre_spectrums_decoys_rand = pp.DecoyParserPassattuto(r'C:\\Users\\Gosia\\Desktop\\GnpsDecoyRandomPeaks').parse_folder()

# Using MatchMS to create decoy spectra
decoys_cond = []
for i, s in enumerate( pre_spectrums_decoys_cond ):
    decoys_cond.append(as_spectrum(s))
    if i and i % 100 == 0:
        print('processed %d'% i)

decoys_rand = []
for i, s in enumerate( pre_spectrums_decoys_rand ):
    decoys_rand.append(as_spectrum(s))
    if i and i % 100 == 0:
        print('processed %d'% i)

processed 100 files
processed 200 files
processed 300 files
processed 400 files
processed 500 files
processed 600 files
processed 700 files
processed 800 files
processed 900 files
processed 1000 files
processed 1100 files
processed 1200 files
processed 1300 files
processed 1400 files
processed 1500 files
processed 1600 files
processed 1700 files
processed 1800 files
processed 1900 files
processed 2000 files
processed 2100 files
processed 2200 files
processed 2300 files
processed 2400 files
processed 2500 files
processed 2600 files
processed 2700 files
processed 2800 files
processed 2900 files
processed 3000 files
processed 3100 files
processed 3200 files


In [ ]:
# This cell is here to save all the query and library spectra to avoid normalization of intensities (for exact replication of passatutto.ipynb)
spectrums_query_cosine = spectrums_query
spectrums_lib_cosine = spectrums_lib
decoys_cond_cosine = decoys_cond
decoys_rand_cosine = decoys_rand

Normalize instensitites for all spectra

In [ ]:
from matchms.filtering import normalize_intensities
# Spec2Vec trained model requires normalizing
spectrums_query = [normalize_intensities(s) for s in spectrums_query]
spectrums_lib = [normalize_intensities(s) for s in spectrums_lib]

decoys_cond = [normalize_intensities(s) for s in decoys_cond]
decoys_rand = [normalize_intensities(s) for s in decoys_rand]


Load pretrained spec2vec model

In [ ]:
path_models = os.path.join(path_data, "trained_models_1")
model_file = os.path.join(path_models, "spec2vec_size_170.model")

model = gensim.models.Word2Vec.load(model_file)
print(model_file)

Create spectrum "documents"

In [ ]:
from spec2vec import Spec2Vec
from spec2vec import SpectrumDocument

documents_query = [SpectrumDocument(s, n_decimals=2) for s in spectrums_query]
documents_lib = [SpectrumDocument(s, n_decimals=2) for s in spectrums_lib]

documents_decoys_cond = [SpectrumDocument(s, n_decimals=2) for s in decoys_cond]
documents_decoys_rand = [SpectrumDocument(s, n_decimals=2) for s in decoys_rand]


Derive embeddings

In [ ]:
from spec2vec.vector_operations import calc_vector

intensity_weighting_power = 0.5
allowed_missing_percentage = 70 # specify the maximum (weighted) fraction of the spectrum that is allowed to be missing

vector_size = model.vector_size
print(f"Embedding vector size: {vector_size}")

embeddings_spec2vec_query = np.zeros((len(documents_query), vector_size), dtype="float")
for i, doc in enumerate(documents_query):
    embeddings_spec2vec_query[i, 0:vector_size] = calc_vector(model, doc,
                                                        intensity_weighting_power,
                                                        allowed_missing_percentage)
embeddings_spec2vec_lib = np.zeros((len(documents_lib), vector_size), dtype="float")
for i, doc in enumerate(documents_lib):
    embeddings_spec2vec_lib[i, 0:vector_size] = calc_vector(model, doc,
                                                        intensity_weighting_power,
                                                        allowed_missing_percentage)

In [ ]:
embeddings_spec2vec_decoys_cond = np.zeros((len(documents_decoys_cond), vector_size), dtype="float")
for i, doc in enumerate(documents_decoys_cond):
    embeddings_spec2vec_decoys_cond[i, 0:vector_size] = calc_vector(model, doc,
                                                        intensity_weighting_power,
                                                        allowed_missing_percentage)
embeddings_spec2vec_decoys_rand = np.zeros((len(documents_decoys_rand), vector_size), dtype="float")
for i, doc in enumerate(documents_decoys_rand):
    embeddings_spec2vec_decoys_rand[i, 0:vector_size] = calc_vector(model, doc,
                                                        intensity_weighting_power,
                                                        allowed_missing_percentage)

In [ ]:
from cosine_calc import get_spec2vec_hits

# Obtain true hits
hits = get_spec2vec_hits( documents_query, documents_lib, model, precursor_tol=3,
                            intensity_weighting_power=intensity_weighting_power,
                            allowed_missing_percentage=allowed_missing_percentage, passatutto=True,
                embeddings_query=embeddings_spec2vec_query, embeddings_library=embeddings_spec2vec_lib)

In [ ]:
def add_pmz(specs):
    from rdkit.Chem import MolFromSmiles, MolToSmiles, MolFromInchi
    from rdkit.Chem.rdMolDescriptors import CalcExactMolWt, CalcMolFormula
    for s in specs:
        s._obj.set('precursor_mz', s._obj.get('parent_mass', 0))
        
add_pmz(documents_query)

In [ ]:
import copy
documents_decoys_cond = []
for v,d in zip(embeddings_spec2vec_decoys_cond,documents_lib):
    decoy_doc = copy.deepcopy(d)
    decoy_doc._obj.set('inchi', 'knockoff')
    decoy_doc._obj.set('vector', v)
    documents_decoys_cond.append(decoy_doc)

# Obtain conditional decoy hits    
hits_decoys_cond = get_spec2vec_hits(documents_query, documents_decoys_cond, decoys=True, model=model, precursor_tol=3, metaKey="precursor_mz",
                                                        intensity_weighting_power=intensity_weighting_power,
                                                        allowed_missing_percentage=allowed_missing_percentage, passatutto=True,
                                    embeddings_query=embeddings_spec2vec_query )


documents_decoys_rand = []
for v,d in zip(embeddings_spec2vec_decoys_rand,documents_lib):
    decoy_doc = copy.deepcopy(d)
    decoy_doc._obj.set('inchi', 'knockoff')
    decoy_doc._obj.set('vector', v)
    documents_decoys_rand.append(decoy_doc)

# Obtain random decoy hits    
hits_decoys_rand = get_spec2vec_hits(documents_query, documents_decoys_rand, decoys=True, model=model, precursor_tol=3, metaKey="precursor_mz",
                                                        intensity_weighting_power=intensity_weighting_power,
                                                        allowed_missing_percentage=allowed_missing_percentage, passatutto=True,
                                    embeddings_query=embeddings_spec2vec_query )

In [ ]:
# Calculate true q-value scores
from q_value_calc import calculate_q_value
q_list_true = calculate_q_value(hits)

In [ ]:
# Calculate estimated q-value scores
q_list_decoys = {}
q_list_decoys['spectrum-based method in Spec2Vec'] = calculate_q_value(hits+hits_decoys_cond,True)
q_list_decoys['naive method in Spec2Vec'] = calculate_q_value(hits+hits_decoys_rand,True)

In [ ]:
from knockoffs import generate_knockoffs

# create new decoys --> knockoffs
knockoff_documents = generate_knockoffs(model,documents_lib,allowed_missing_percentage=allowed_missing_percentage,n_components=3, diagonal_matrix=110, embeddings=embeddings_spec2vec_lib)
hits_knockoffs = get_spec2vec_hits(documents_query, knockoff_documents, decoys=True, model=model, precursor_tol=3, metaKey="precursor_mz",
                                   intensity_weighting_power=intensity_weighting_power,
                                   allowed_missing_percentage=allowed_missing_percentage, passatutto=True, embeddings_query=embeddings_spec2vec_query)
q_list_decoys["knockoff method in Spec2Vec"] = calculate_q_value(hits+hits_knockoffs,True)

In [ ]:
import plot_q_vals
plot_q_vals.plot_q_vals(q_list_true, {k:v for k,v in q_list_decoys.items() if 'Spec2Vec' in k} )

In [ ]:
# Calculate cosine similarity for target-query match
from cosine_calc import get_hits
hits_cosine = get_hits(spectrums_query_cosine, spectrums_lib_cosine, 3, cosine_tol=0.005, passatutto=True)

In [ ]:
from rdkit.Chem import MolFromSmiles, MolToSmiles, MolFromInchi
from rdkit.Chem.rdMolDescriptors import CalcExactMolWt, CalcMolFormula
def add_pmz(specs):
    for s in specs:
        s.set('precursor_mz', s.get('parent_mass', 0))

add_pmz(spectrums_query_cosine)

In [ ]:
# Calculate cosine similarity for query-decoy match
from cosine_calc import get_hits
hits_decoys_cond_cosine = get_hits(spectrums_query_cosine, decoys_cond_cosine, 3, "precursor_mz", cosine_tol=0.005, decoys=True, passatutto=True)
hits_decoys_rand_cosine = get_hits(spectrums_query_cosine, decoys_rand_cosine, 3, "precursor_mz", cosine_tol=0.005, decoys=True, passatutto=True)

In [ ]:
#lengths of all cosine hits
print(len(hits_cosine))
print(len(hits_decoys_cond_cosine))
print(len(hits_decoys_rand_cosine))

In [ ]:
# Calculate true q-value scores for comparison with contemporary cosince
from q_value_calc import calculate_q_value
q_list_true_cosine = calculate_q_value(hits_cosine)

In [ ]:
# Calculate estimated q-value scores (cosine comparison again)
q_list_decoys['spectrum-based method'] = calculate_q_value(hits_cosine+hits_decoys_cond_cosine,True)
q_list_decoys['naive method'] = calculate_q_value(hits_cosine+hits_decoys_rand_cosine,True)

Custom plotting to combine Spec2Vec and Cosine results

In [ ]:
import pylab as plt

def combine_true_est(q_val_true, q_val_est):
    res = []
    q_idx = 0
    for q_e, _, score in q_val_est:
        while q_idx < len(q_val_true) - 1 and q_val_true[q_idx + 1][2] >= score:
            q_idx += 1
        res.append((score, q_val_true[q_idx][0], q_e))
    return res


to_plot = {}
for label, q_val_est in q_list_decoys.items():
    if 'Spec2Vec' in label:
        to_plot[label] = list(zip(*combine_true_est(q_list_true, q_val_est)))[1], list(zip(*combine_true_est(q_list_true, q_val_est)))[2]
    else:
        to_plot[label] = list(zip(*combine_true_est(q_list_true_cosine, q_val_est)))[1], list(zip(*combine_true_est(q_list_true_cosine, q_val_est)))[2]
        
fig, ax = plt.subplots(figsize=(7, 7))

# print CosineGreedy-based decoys first, to preserve colours from passatutto.ipynb
label = 'spectrum-based method'
tru,est = to_plot[label]
plt.plot(tru, est, label=label)
label = 'naive method'
tru,est = to_plot[label]
plt.plot(tru, est, label=label)

for label, (tru, est) in to_plot.items():
    if 'Spec2Vec' in label:
        plt.plot(tru, est, label=label)
plt.plot([0, 0.5], [0, 0.5], 'k--')
plt.xlabel('True q-value')
plt.ylabel('Estimated q-value')
plt.legend()
plt.grid()


In [ ]:
# Error statistics
for label, (tru, est) in to_plot.items():
    vals = [ abs(e-t) for t,e in zip(tru,est) ]

    print("For", label)
    print("avg error:", sum(vals) / len(vals) )
    vals = [ abs(e-t) for t,e in zip(tru,est) if t<0.5 ]
    print("avg error for true q-value < 0.5:", sum(vals) / len(vals) )
